In [1]:
!pip install geopy

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import datetime

In [3]:
# Define the path to the folder
folder_path = Path("C:/Users/crazy/OneDrive - The City College of New York/DSE I2100 - Applied Machine Learning and Data Mining/Project")
csv_file = folder_path.glob("*.csv").__next__()

# Load CSV file into DataFrame
df = pd.read_csv(csv_file)


D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## The Following code replaces 0 value rows with NaN for 'LATITUDE' & 'LONGITUDE' columns:

In [4]:
# verify we have valid coordinates in the data

# Filter the DataFrame for latitude and longitude values less than 1 and greater than -1
filtered_df = df[(df['LATITUDE'] < 1) & (df['LATITUDE'] > -1) | (df['LONGITUDE'] < 1) & (df['LONGITUDE'] > -1)]
print(filtered_df[['LATITUDE','LONGITUDE']])

         LATITUDE  LONGITUDE
44            0.0        0.0
47            0.0        0.0
212           0.0        0.0
252           0.0        0.0
537           0.0        0.0
...           ...        ...
2066947       0.0        0.0
2067378       0.0        0.0
2069047       0.0        0.0
2069552       0.0        0.0
2070021       0.0        0.0

[4350 rows x 2 columns]


In [5]:
# Approximate coordinates for New York City:
# Maximum Latitude: 40.9176 (Northernmost point of the Bronx)
# Minimum Latitude: 40.4774 (Southernmost point of Staten Island)
# Maximum Longitude: -73.7004 (Easternmost point of Queens)
# Minimum Longitude: -74.2591 (Westernmost point of Staten Island)

# Define the maximum and minimum values for latitude and longitude
max_latitude = 40.9176
min_latitude = 40.4774
max_longitude = -73.7004
min_longitude = -74.2591

# Filter the DataFrame based on the conditions for latitude and longitude
invalid_latitudes = (df['LATITUDE'] > max_latitude) | (df['LATITUDE'] < min_latitude)
invalid_longitudes = (df['LONGITUDE'] > max_longitude) | (df['LONGITUDE'] < min_longitude)

# Replace the values with NaN where the conditions are not met
df.loc[invalid_latitudes, 'LATITUDE'] = np.nan
df.loc[invalid_longitudes, 'LONGITUDE'] = np.nan

In [6]:
test_filtered_df = df[(df['LATITUDE'] < 1) & (df['LATITUDE'] > -1) | (df['LONGITUDE'] < 1) & (df['LONGITUDE'] > -1)]
print(test_filtered_df[['LATITUDE','LONGITUDE']])

Empty DataFrame
Columns: [LATITUDE, LONGITUDE]
Index: []


## The following code creates a function to replace NaN values in the 'BOROUGH', 'ZIP CODE', 'ON STREET NAME' with actual values:

In [7]:
# Function to find values for BOROUGH, ZIP CODE & ON STREET NAME using coordinates
geolocator = Nominatim(user_agent="accident_severity_prediction")

def reverse_geocode(lat, lon, borough, zipcode, street_name):
    
    if np.isnan(lat) or np.isnan(lon) or bool(borough) == bool(zipcode) == bool(street_name) == "True":
        #return [None, None, None]
        return [np.nan, np.nan, np.nan]
                                                                                                 
    else:
        
        try:
            location = geolocator.reverse((lat, lon))
            address = location.raw
            borough = address.get('address', {}).get('suburb')
            zipcode = address.get('address', {}).get('postcode')
            on_street_name = address.get('address', {}).get('road')
            return [borough, zipcode, on_street_name]

        except:
            #return [None, None, None]
            return [np.nan, np.nan, np.nan]

In [ ]:
# Apply the reverse_geocode function to each row of the dataframe
new_values = df.apply(lambda row: reverse_geocode(row['LATITUDE'], row['LONGITUDE'], row['BOROUGH'], row['ZIP CODE'], row['ON STREET NAME']), axis=1, result_type='expand')

# Replace NaN values in 'borough', 'zipcode', and 'address' columns with the corresponding values from the reverse_geocode function
df['BOROUGH'].fillna(new_values[0], inplace=True)
df['ZIP CODE'].fillna(new_values[1], inplace=True)
df['ON STREET NAME'].fillna(new_values[2], inplace=True)

The function does not remove any existing values, only adds values to the 'BOROUGH', 'ZIP CODE', 'ON STREET NAME' columns.

Issue with trying to run big dataset with function, maybe add code to prevent iteration when borough, zipcode and onstreetname are not NaN

1. Also, need to complete and make sure new values match dataset, like borough data
2. convert zipcode data to float64
3. convert borough and street name to str object

## Renaming duplicate values for "BOROUGH" column:

In [ ]:
# Get the counts of unique values in the "BOROUGH" column
borough_counts = df['BOROUGH'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'BOROUGH' column:")
print(borough_counts)

# Calculate the total sum of the counts
total_sum = borough_counts.sum()
print("Total sum of counts:", total_sum)


In [ ]:
# Convert all values in the "BOROUGH" column to lowercase
df['BOROUGH'] = df['BOROUGH'].str.lower()

# Replace specific values in the "BOROUGH" column
df['BOROUGH'].replace({'the bronx': 'bronx', 'queens county': 'queens'}, inplace=True)

In [ ]:
# Get the counts of unique values in the "BOROUGH" column
borough_counts = df['BOROUGH'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'BOROUGH' column:")
print(borough_counts)

# Calculate the total sum of the counts
total_sum = borough_counts.sum()
print("Total sum of counts:", total_sum)

## Converting zipcodes to string objects

In [ ]:
# Get the counts of unique values in the "BOROUGH" column
zipcode_counts = df['ZIP CODE'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(zipcode_counts)

# Calculate the total sum of the counts
total_sum = zipcode_counts.sum()
print("Total sum of counts:", total_sum)

In [ ]:
# Convert 'ZIP CODE' column to float, handling NaN values
df['ZIP CODE'] = pd.to_numeric(df['ZIP CODE'], errors='coerce')

# Convert 'ZIP CODE' column to int (this will automatically drop any decimal points)
df['ZIP CODE'] = df['ZIP CODE'].astype(float).astype('Int64')  # Convert to Int64 to handle NaN and preserve integer dtype

# Convert 'ZIP CODE' column back to string
df['ZIP CODE'] = df['ZIP CODE'].astype(str)

In [ ]:
# Get the counts of unique values in the "BOROUGH" column
zipcode_counts = df['ZIP CODE'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(zipcode_counts)

# Calculate the total sum of the counts
total_sum = zipcode_counts.sum()
print("Total sum of counts:", total_sum)

## Converting street names to lowercase string objects

In [ ]:
# Get the counts of unique values in the "BOROUGH" column
street_counts = df['ON STREET NAME'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(street_counts)

# Calculate the total sum of the counts
total_sum = street_counts.sum()
print("Total sum of counts:", total_sum)

In [ ]:
# Convert all values in the "BOROUGH" column to lowercase
df['ON STREET NAME'] = df['ON STREET NAME'].str.lower()


In [ ]:
# Get the counts of unique values in the "BOROUGH" column
street_counts = df['ON STREET NAME'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(street_counts)

# Calculate the total sum of the counts
total_sum = street_counts.sum()
print("Total sum of counts:", total_sum)

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('Preprocessed_Data.csv', index=False)